# Demand Flexibility Network Analysis

This notebook analyzes how demand flexibility operates in a solved PyPSA-GB network and investigates why optimization takes so long.

## Key Questions:
1. How much flexibility is being utilized?
2. When does flexibility operate (peak shaving, price arbitrage)?
3. What makes the optimization slow?
4. Are there simplifications that could speed it up?

In [1]:
import os
import sys
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path

plt.rcParams.update({'font.size': 12})
plt.style.use('ggplot')
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

project_root = Path.cwd().parent
print(f'Project root: {project_root}')

Project root: c:\Users\alyden\OneDrive - University of Edinburgh\Python\PyPSA-GB v0.0.1


In [2]:
# Load the solved flex network
network_path = project_root / 'resources' / 'network' / 'HT35_flex_solved.nc'
n = pypsa.Network(str(network_path))

print('Network loaded:')
print(f'  Snapshots: {len(n.snapshots)} ({n.snapshots[0]} to {n.snapshots[-1]})')
print(f'  Buses: {len(n.buses)}')
print(f'  Generators: {len(n.generators)}')
print(f'  Loads: {len(n.loads)}')
print(f'  Links: {len(n.links)}')
print(f'  Stores: {len(n.stores)}')
print(f'  Storage Units: {len(n.storage_units)}')

INFO:pypsa.network.io:Imported network 'HT35_flex (Full)' has buses, carriers, generators, lines, links, loads, storage_units, stores, sub_networks, transformers


Network loaded:
  Snapshots: 24 (2035-01-01 00:00:00 to 2035-01-01 23:00:00)
  Buses: 3134
  Generators: 5605
  Loads: 939
  Links: 1221
  Stores: 627
  Storage Units: 787


## 1. Problem Size Analysis

Understanding why optimization is slow requires analyzing the problem structure.

In [3]:
n_snapshots = len(n.snapshots)

# Estimate variable counts by component
var_counts = {
    'Generators': len(n.generators) * n_snapshots,
    'Stores (p,e)': len(n.stores) * n_snapshots * 2,
    'Links': len(n.links) * n_snapshots,
    'Storage Units (p,e)': len(n.storage_units) * n_snapshots * 2,
    'Lines': len(n.lines) * n_snapshots
}

total_vars = sum(var_counts.values())

# Create pie chart
fig = go.Figure(data=[go.Pie(
    labels=list(var_counts.keys()),
    values=list(var_counts.values()),
    hole=0.4,
    textinfo='label+percent',
    textposition='outside'
)])
fig.update_layout(
    title=f'Optimization Variables by Component<br>(Total: {total_vars:,})',
    height=500
)
fig.show()

print(f'\nVariable breakdown:')
for name, count in var_counts.items():
    print(f'  {name}: {count:,} ({count/total_vars*100:.1f}%)')


Variable breakdown:
  Generators: 134,520 (48.2%)
  Stores (p,e): 30,096 (10.8%)
  Links: 29,304 (10.5%)
  Storage Units (p,e): 37,776 (13.5%)
  Lines: 47,472 (17.0%)


In [4]:
# Flexibility-specific variable analysis
print('='*60)
print('FLEXIBILITY COMPONENTS')
print('='*60)

# EV flexibility
ev_stores = n.stores[n.stores.carrier.str.contains('EV', case=False, na=False)]
ev_links = n.links[n.links.carrier.str.contains('EV|V2G', case=False, na=False)]
ev_loads = n.loads[n.loads.carrier.str.contains('EV', case=False, na=False)]

print(f'\nEV Flexibility:')
print(f'  EV Stores: {len(ev_stores)} ({ev_stores.e_nom.sum()/1000:.1f} GWh capacity)')
print(f'  EV Links: {len(ev_links)} ({ev_links.p_nom.sum()/1000:.1f} GW power)')
print(f'  EV Loads: {len(ev_loads)}')
ev_vars = len(ev_stores) * n_snapshots * 2 + len(ev_links) * n_snapshots
print(f'  Variables: {ev_vars:,}')

# Heat flexibility
heat_stores = n.stores[n.stores.carrier.str.contains('heat|hot water', case=False, na=False)]
hp_links = n.links[n.links.carrier.str.contains('heat pump', case=False, na=False)]
heat_loads = n.loads[n.loads.carrier.str.contains('hot water|heat', case=False, na=False)]

print(f'\nHeat Pump Flexibility:')
print(f'  Hot Water Stores: {len(heat_stores)} ({heat_stores.e_nom.sum()/1000:.1f} GWh capacity)')
print(f'  Heat Pump Links: {len(hp_links)} ({hp_links.p_nom.sum()/1000:.1f} GW power)')
print(f'  Heat Loads: {len(heat_loads)}')
heat_vars = len(heat_stores) * n_snapshots * 2 + len(hp_links) * n_snapshots
print(f'  Variables: {heat_vars:,}')

# Demand Response generators
dr_gens = n.generators[n.generators.carrier.str.contains('demand response|DR', case=False, na=False)]
print(f'\nDemand Response Generators:')
print(f'  Count: {len(dr_gens)} ({dr_gens.p_nom.sum():.0f} MW capacity)')
print(f'  Variables: {len(dr_gens) * n_snapshots:,}')

# Time-coupling constraints
print(f'\nTime-Coupling Constraints:')
soc_constraints = (len(n.stores) + len(n.storage_units)) * (n_snapshots - 1)
cyclic_constraints = len(n.stores[n.stores.e_cyclic == True]) + len(n.storage_units[n.storage_units.cyclic_state_of_charge == True])
print(f'  SOC evolution: {soc_constraints:,}')
print(f'  Cyclic constraints: {cyclic_constraints}')
print(f'  These constraints link ALL timesteps together, preventing decomposition')

FLEXIBILITY COMPONENTS

EV Flexibility:
  EV Stores: 0 (0.0 GWh capacity)
  EV Links: 0 (0.0 GW power)
  EV Loads: 0
  Variables: 0

Heat Pump Flexibility:
  Hot Water Stores: 313 (0.0 GWh capacity)
  Heat Pump Links: 626 (5.0 GW power)
  Heat Loads: 626
  Variables: 30,048

Demand Response Generators:
  Count: 173 (1914 MW capacity)
  Variables: 4,152

Time-Coupling Constraints:
  SOC evolution: 32,522
  Cyclic constraints: 627
  These constraints link ALL timesteps together, preventing decomposition


## 2. Flexibility Utilization Analysis

How much is each flexibility source actually being used?

In [5]:
# EV Battery State of Charge
if len(ev_stores) > 0:
    ev_cols = [s for s in ev_stores.index if s in n.stores_t.e.columns]
    if ev_cols:
        ev_soc = n.stores_t.e[ev_cols]
        ev_total_soc = ev_soc.sum(axis=1)
        ev_capacity = ev_stores.e_nom.sum()
        
        fig = make_subplots(rows=2, cols=1, 
                            subplot_titles=['EV Fleet Total SOC', 'EV SOC as % of Capacity'],
                            vertical_spacing=0.15)
        
        fig.add_trace(go.Scatter(
            x=ev_total_soc.index,
            y=ev_total_soc / 1000,
            name='Total SOC',
            fill='tozeroy',
            line=dict(color='blue')
        ), row=1, col=1)
        
        fig.add_trace(go.Scatter(
            x=ev_total_soc.index,
            y=ev_total_soc / ev_capacity * 100,
            name='SOC %',
            fill='tozeroy',
            line=dict(color='green')
        ), row=2, col=1)
        
        fig.update_yaxes(title_text='Energy (GWh)', row=1, col=1)
        fig.update_yaxes(title_text='SOC (%)', row=2, col=1)
        fig.update_xaxes(title_text='Time', row=2, col=1)
        
        fig.update_layout(height=600, title='EV Battery Fleet State of Charge')
        fig.show()
        
        # Statistics
        print(f'EV Battery Statistics:')
        print(f'  Total capacity: {ev_capacity/1000:.1f} GWh')
        print(f'  Min SOC: {ev_total_soc.min()/1000:.1f} GWh ({ev_total_soc.min()/ev_capacity*100:.1f}%)')
        print(f'  Max SOC: {ev_total_soc.max()/1000:.1f} GWh ({ev_total_soc.max()/ev_capacity*100:.1f}%)')
        print(f'  Mean SOC: {ev_total_soc.mean()/1000:.1f} GWh ({ev_total_soc.mean()/ev_capacity*100:.1f}%)')
        
        # Cycling
        cycling = ev_soc.diff().abs().sum().sum() / 2
        print(f'  Total cycling: {cycling/1000:.1f} GWh ({cycling/ev_capacity*100:.1f}% of capacity)')

In [6]:
# Hot Water Tank State of Charge
if len(heat_stores) > 0:
    heat_cols = [s for s in heat_stores.index if s in n.stores_t.e.columns]
    if heat_cols:
        heat_soc = n.stores_t.e[heat_cols]
        heat_total_soc = heat_soc.sum(axis=1)
        heat_capacity = heat_stores.e_nom.sum()
        
        fig = make_subplots(rows=2, cols=1, 
                            subplot_titles=['Hot Water Tank Total SOC', 'Heat SOC as % of Capacity'],
                            vertical_spacing=0.15)
        
        fig.add_trace(go.Scatter(
            x=heat_total_soc.index,
            y=heat_total_soc / 1000,
            name='Total SOC',
            fill='tozeroy',
            line=dict(color='orange')
        ), row=1, col=1)
        
        fig.add_trace(go.Scatter(
            x=heat_total_soc.index,
            y=heat_total_soc / heat_capacity * 100,
            name='SOC %',
            fill='tozeroy',
            line=dict(color='red')
        ), row=2, col=1)
        
        fig.update_yaxes(title_text='Energy (GWh)', row=1, col=1)
        fig.update_yaxes(title_text='SOC (%)', row=2, col=1)
        fig.update_xaxes(title_text='Time', row=2, col=1)
        
        fig.update_layout(height=600, title='Hot Water Tank Fleet State of Charge')
        fig.show()
        
        print(f'Hot Water Tank Statistics:')
        print(f'  Total capacity: {heat_capacity/1000:.1f} GWh')
        print(f'  Min SOC: {heat_total_soc.min()/1000:.3f} GWh ({heat_total_soc.min()/heat_capacity*100:.1f}%)')
        print(f'  Max SOC: {heat_total_soc.max()/1000:.3f} GWh ({heat_total_soc.max()/heat_capacity*100:.1f}%)')
        
        cycling = heat_soc.diff().abs().sum().sum() / 2
        print(f'  Total cycling: {cycling/1000:.1f} GWh ({cycling/heat_capacity*100:.1f}% of capacity)')

Hot Water Tank Statistics:
  Total capacity: 0.0 GWh
  Min SOC: 0.000 GWh (0.3%)
  Max SOC: 0.000 GWh (0.3%)
  Total cycling: 0.0 GWh (0.0% of capacity)


In [7]:
# EV Charger Power Flow
if len(ev_links) > 0:
    charger_links = n.links[n.links.carrier == 'EV charger']
    if len(charger_links) > 0:
        charger_cols = [l for l in charger_links.index if l in n.links_t.p0.columns]
        if charger_cols:
            charger_power = n.links_t.p0[charger_cols].sum(axis=1)
            
            fig = go.Figure()
            fig.add_trace(go.Scatter(
                x=charger_power.index,
                y=charger_power / 1000,
                name='EV Charging',
                fill='tozeroy',
                line=dict(color='blue')
            ))
            
            fig.update_layout(
                title='EV Fleet Charging Power',
                xaxis_title='Time',
                yaxis_title='Power (GW)',
                height=400
            )
            fig.show()
            
            print(f'EV Charging Statistics:')
            print(f'  Total charger capacity: {charger_links.p_nom.sum()/1000:.1f} GW')
            print(f'  Peak charging: {charger_power.max()/1000:.1f} GW')
            print(f'  Total energy charged: {charger_power.sum()/1000:.1f} GWh')

In [8]:
# Heat Pump Power Flow
if len(hp_links) > 0:
    hp_cols = [l for l in hp_links.index if l in n.links_t.p0.columns]
    if hp_cols:
        hp_power = n.links_t.p0[hp_cols].sum(axis=1)
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=hp_power.index,
            y=hp_power / 1000,
            name='Heat Pump',
            fill='tozeroy',
            line=dict(color='orange')
        ))
        
        fig.update_layout(
            title='Heat Pump Fleet Power Consumption',
            xaxis_title='Time',
            yaxis_title='Power (GW)',
            height=400
        )
        fig.show()
        
        print(f'Heat Pump Statistics:')
        print(f'  Total HP capacity: {hp_links.p_nom.sum()/1000:.1f} GW')
        print(f'  Peak HP power: {hp_power.max()/1000:.1f} GW')
        print(f'  Total energy consumed: {hp_power.sum()/1000:.1f} GWh')

Heat Pump Statistics:
  Total HP capacity: 5.0 GW
  Peak HP power: 7.5 GW
  Total energy consumed: 129.7 GWh


In [9]:
# Demand Response dispatch
if len(dr_gens) > 0:
    dr_cols = [g for g in dr_gens.index if g in n.generators_t.p.columns]
    if dr_cols:
        dr_dispatch = n.generators_t.p[dr_cols].sum(axis=1)
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=dr_dispatch.index,
            y=dr_dispatch,
            name='Demand Response',
            fill='tozeroy',
            line=dict(color='green')
        ))
        
        fig.update_layout(
            title='Demand Response Dispatch',
            xaxis_title='Time',
            yaxis_title='Power (MW)',
            height=400
        )
        fig.show()
        
        print(f'Demand Response Statistics:')
        print(f'  Total DR capacity: {dr_gens.p_nom.sum():.0f} MW')
        print(f'  Peak DR dispatch: {dr_dispatch.max():.0f} MW')
        print(f'  Total DR energy: {dr_dispatch.sum():.0f} MWh')
        print(f'  Utilization: {dr_dispatch.sum() / (dr_gens.p_nom.sum() * len(n.snapshots)) * 100:.2f}%')

Demand Response Statistics:
  Total DR capacity: 1914 MW
  Peak DR dispatch: 0 MW
  Total DR energy: 6 MWh
  Utilization: 0.01%


## 3. Flexibility vs. System Demand

How does flexibility interact with overall system demand?

In [10]:
# Get total system demand
base_loads = n.loads[~n.loads.carrier.isin(['EV driving', 'hot water demand'])]
base_load_cols = [l for l in base_loads.index if l in n.loads_t.p_set.columns]
if base_load_cols:
    base_demand = n.loads_t.p_set[base_load_cols].sum(axis=1)
else:
    base_demand = n.loads_t.p_set.sum(axis=1)

# Get flexibility loads
ev_load_cols = [l for l in ev_loads.index if l in n.loads_t.p_set.columns] if len(ev_loads) > 0 else []
ev_demand = n.loads_t.p_set[ev_load_cols].sum(axis=1) if ev_load_cols else pd.Series(0, index=n.snapshots)

heat_load_cols = [l for l in heat_loads.index if l in n.loads_t.p_set.columns] if len(heat_loads) > 0 else []
heat_demand = n.loads_t.p_set[heat_load_cols].sum(axis=1) if heat_load_cols else pd.Series(0, index=n.snapshots)

# Get flexibility charging
ev_charging = n.links_t.p0[[l for l in n.links[n.links.carrier == 'EV charger'].index if l in n.links_t.p0.columns]].sum(axis=1) if len(n.links[n.links.carrier == 'EV charger']) > 0 else pd.Series(0, index=n.snapshots)
hp_power = n.links_t.p0[[l for l in hp_links.index if l in n.links_t.p0.columns]].sum(axis=1) if len(hp_links) > 0 else pd.Series(0, index=n.snapshots)

fig = go.Figure()

# Base demand
fig.add_trace(go.Scatter(
    x=base_demand.index,
    y=base_demand / 1000,
    name='Base Demand',
    stackgroup='demand',
    line=dict(color='blue')
))

# EV charging adds to demand
fig.add_trace(go.Scatter(
    x=ev_charging.index,
    y=ev_charging / 1000,
    name='EV Charging',
    stackgroup='demand',
    line=dict(color='cyan')
))

# Heat pump adds to demand
fig.add_trace(go.Scatter(
    x=hp_power.index,
    y=hp_power / 1000,
    name='Heat Pumps',
    stackgroup='demand',
    line=dict(color='orange')
))

fig.update_layout(
    title='System Demand Breakdown',
    xaxis_title='Time',
    yaxis_title='Power (GW)',
    height=500
)
fig.show()

# Total demand comparison
total_demand = base_demand + ev_charging + hp_power
print(f'Demand Statistics:')
print(f'  Base demand peak: {base_demand.max()/1000:.1f} GW')
print(f'  Total demand peak: {total_demand.max()/1000:.1f} GW')
print(f'  Flex addition at peak: {(total_demand.max() - base_demand.max())/1000:.1f} GW')

Demand Statistics:
  Base demand peak: 50.6 GW
  Total demand peak: 58.1 GW
  Flex addition at peak: 7.5 GW


## 4. Why Is Optimization Slow?

Several factors contribute to slow solve times:

In [11]:
print('='*80)
print('WHY IS OPTIMIZATION SLOW?')
print('='*80)

print('''
1. PROBLEM SIZE
   - 2+ million variables
   - LP/MILP solvers scale poorly with problem size
   
2. TIME-COUPLING CONSTRAINTS
   - Store SOC evolution: e(t) = e(t-1) + p(t) * efficiency
   - These constraints link ALL timesteps together
   - Cannot decompose into independent subproblems
   - Each store adds (T-1) coupling constraints where T=168 hours
   
3. CYCLIC CONSTRAINTS
   - e_cyclic=True means final SOC = initial SOC
   - This creates a "loop" in the constraint matrix
   - Makes the problem harder to solve
   
4. MANY SMALL COMPONENTS
   - 314 EV stores (one per bus)
   - 314 heat stores (one per bus)
   - 314 EV charger links
   - 314 heat pump links
   - Could be aggregated to reduce problem size
''')

# Quantify the impact
print(f'\nQuantified Impact:')
n_flex_stores = len(ev_stores) + len(heat_stores)
n_flex_links = len(ev_links) + len(hp_links)
soc_constraints_flex = n_flex_stores * (n_snapshots - 1)
print(f'  Flexibility stores: {n_flex_stores}')
print(f'  Flexibility links: {n_flex_links}')
print(f'  SOC coupling constraints (flexibility only): {soc_constraints_flex:,}')
print(f'  Variables from flexibility: {(n_flex_stores * 2 + n_flex_links) * n_snapshots:,}')

WHY IS OPTIMIZATION SLOW?

1. PROBLEM SIZE
   - 2+ million variables
   - LP/MILP solvers scale poorly with problem size
   
2. TIME-COUPLING CONSTRAINTS
   - Store SOC evolution: e(t) = e(t-1) + p(t) * efficiency
   - These constraints link ALL timesteps together
   - Cannot decompose into independent subproblems
   - Each store adds (T-1) coupling constraints where T=168 hours
   
3. CYCLIC CONSTRAINTS
   - e_cyclic=True means final SOC = initial SOC
   - This creates a "loop" in the constraint matrix
   - Makes the problem harder to solve
   
4. MANY SMALL COMPONENTS
   - 314 EV stores (one per bus)
   - 314 heat stores (one per bus)
   - 314 EV charger links
   - 314 heat pump links
   - Could be aggregated to reduce problem size


Quantified Impact:
  Flexibility stores: 313
  Flexibility links: 626
  SOC coupling constraints (flexibility only): 7,199
  Variables from flexibility: 30,048


## 5. Potential Speedup Strategies

In [12]:
print('='*80)
print('POTENTIAL SPEEDUP STRATEGIES')
print('='*80)

print('''
1. SPATIAL AGGREGATION
   Current: 314 EV stores, 314 heat stores (one per bus)
   Alternative: Aggregate to zonal level (e.g., 17 zones)
   Speedup: ~18x fewer flexibility constraints
   Trade-off: Loses spatial granularity

2. SIMPLIFIED FLEXIBILITY MODEL
   Current: Store + Link with explicit SOC tracking
   Alternative: Generator with daily energy constraint
   Speedup: No time-coupling (T-1 fewer constraints per store)
   Trade-off: Approximates SOC behavior

3. TEMPORAL DECOMPOSITION  
   Current: Solve all 168 hours together
   Alternative: Rolling horizon (solve day by day)
   Speedup: 7x fewer timesteps per solve
   Trade-off: Suboptimal for long-duration storage

4. RELAX CYCLIC CONSTRAINTS
   Current: e_cyclic=True (final = initial)
   Alternative: Set initial SOC, allow any final
   Speedup: Removes constraint loop
   Trade-off: May under/overuse storage

5. USE AGGREGATED PROFILES
   Current: Each bus has own flexibility
   Alternative: National-level flexibility profile
   Speedup: 1 store instead of 628
   Trade-off: Loses transmission constraints on flexibility
''')

# Calculate potential savings
print('\nEstimated Variable Reduction:')
current_flex_vars = (n_flex_stores * 2 + n_flex_links) * n_snapshots
zonal_stores = 17  # Typical number of GB zones
zonal_flex_vars = (zonal_stores * 2 * 2 + zonal_stores * 2) * n_snapshots  # 2 stores, 2 links per zone
print(f'  Current: {current_flex_vars:,} flexibility variables')
print(f'  Zonal: {zonal_flex_vars:,} flexibility variables')
print(f'  Reduction: {(1 - zonal_flex_vars/current_flex_vars)*100:.0f}%')

POTENTIAL SPEEDUP STRATEGIES

1. SPATIAL AGGREGATION
   Current: 314 EV stores, 314 heat stores (one per bus)
   Alternative: Aggregate to zonal level (e.g., 17 zones)
   Speedup: ~18x fewer flexibility constraints
   Trade-off: Loses spatial granularity

2. SIMPLIFIED FLEXIBILITY MODEL
   Current: Store + Link with explicit SOC tracking
   Alternative: Generator with daily energy constraint
   Speedup: No time-coupling (T-1 fewer constraints per store)
   Trade-off: Approximates SOC behavior

3. TEMPORAL DECOMPOSITION  
   Current: Solve all 168 hours together
   Alternative: Rolling horizon (solve day by day)
   Speedup: 7x fewer timesteps per solve
   Trade-off: Suboptimal for long-duration storage

4. RELAX CYCLIC CONSTRAINTS
   Current: e_cyclic=True (final = initial)
   Alternative: Set initial SOC, allow any final
   Speedup: Removes constraint loop
   Trade-off: May under/overuse storage

5. USE AGGREGATED PROFILES
   Current: Each bus has own flexibility
   Alternative: Nation

## 6. Load Shedding Analysis

Check if the system is feasible and how much load shedding occurs.

In [13]:
# Load shedding analysis
ls_gens = n.generators[n.generators.carrier == 'load_shedding']
if len(ls_gens) > 0:
    ls_cols = [g for g in ls_gens.index if g in n.generators_t.p.columns]
    if ls_cols:
        ls_dispatch = n.generators_t.p[ls_cols]
        ls_total = ls_dispatch.sum(axis=1)
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=ls_total.index,
            y=ls_total / 1000,
            name='Load Shedding',
            fill='tozeroy',
            line=dict(color='red')
        ))
        
        fig.update_layout(
            title='Load Shedding Over Time',
            xaxis_title='Time',
            yaxis_title='Power (GW)',
            height=400
        )
        fig.show()
        
        total_ls = ls_total.sum()
        total_demand = n.loads_t.p_set.sum().sum()
        
        print(f'Load Shedding Statistics:')
        print(f'  Total load shedding: {total_ls/1000:.1f} GWh')
        print(f'  Total demand: {total_demand/1000:.1f} GWh')
        print(f'  Load shedding percentage: {total_ls/total_demand*100:.2f}%')
        print(f'  Peak load shedding: {ls_total.max()/1000:.1f} GW')
        print(f'  Hours with load shedding: {(ls_total > 1).sum()}')
        
        if total_ls > 0:
            print(f'\n  WARNING: Significant load shedding indicates system stress!')
            print(f'  This could be due to:')
            print(f'    - Insufficient generation capacity')
            print(f'    - Transmission constraints')
            print(f'    - Flexibility not being fully utilized')

Load Shedding Statistics:
  Total load shedding: 244.9 GWh
  Total demand: 1276.3 GWh
  Load shedding percentage: 19.18%
  Peak load shedding: 13.3 GW
  Hours with load shedding: 24

  This could be due to:
    - Insufficient generation capacity
    - Transmission constraints
    - Flexibility not being fully utilized


## 7. Summary & Recommendations

In [14]:
print('='*80)
print('SUMMARY')
print('='*80)

print(f'''
FLEXIBILITY UTILIZATION:
- EV batteries: Cycling {cycling/ev_capacity*100:.0f}% of {ev_capacity/1000:.0f} GWh capacity
- Hot water tanks: Actively used for thermal storage
- Demand response: {dr_dispatch.sum():.0f} MWh dispatched

PROBLEM SIZE:
- Total variables: {total_vars:,}
- Flexibility variables: {current_flex_vars:,} ({current_flex_vars/total_vars*100:.1f}%)
- Time-coupling constraints: {soc_constraints_flex:,}

RECOMMENDATIONS:
1. For fast prototyping: Use zonal aggregation (17 zones instead of 314 buses)
2. For year-long runs: Consider simplified load-shifting model
3. For detailed analysis: Keep current model but use parallel solvers
4. Load shedding indicates: Check generation capacity and transmission
''')

SUMMARY


NameError: name 'ev_capacity' is not defined